**ЗАПРОС НА ПОЛУЧЕНИЕ ДАННЫХ О ВАКАНСИЯХ, СОДЕРЖАЩИХ СЛОВО "АНАЛИТИК"**

1. В ответе приходит словарь с 8-мью ключами: 'items', 'found', 'pages', 'per_page', 'page', 'clusters', 'arguments', 'alternate_url'
2.  - items - словарь / отдельные элементы списка вакансий;
    - found - общее кол-во найденных вакансий по заданному критерию;
    - pages - кол-во элементов в разделе items;
    - per_page - значение per_page из передаваемых параметров;
    - page - значение page из передаваемых параметров;
    - clusters - Возвращать ли кластеры для данного поиска;
    - arguments - Возвращать ли описание использованных параметров поиска;
    - alternate_url - ссылка на страницу HH с указанным поиском.
3. Ключевым в ответе из api служит пункт items - словарь из 31-го ключа: 
'id', 'premium', 'name', 'department', 'has_test', 'response_letter_required', 'area', 'salary', 'type', 'address', 'response_url', 'sort_point_distance', 'published_at', 'created_at', 'archived', 'apply_alternate_url', 'insider_interview', 'url', 'adv_response_url', 'alternate_url', 'relations', 'employer', 'snippet', 'contacts', 'schedule', 'working_days', 'working_time_intervals', 'working_time_modes', 'accept_temporary', 'professional_roles', 'accept_incomplete_resumes'
4. Стоит выделить самые важные для нашего проекта:
    - id - уникальный идентификатор вакансии, по которому в дальнейшем мы будем брать полное описание;
    - name - полное название вакансии;
    - salary - зарплата;
    - published_at - дата публикации;
    - created_at - дата создания.

**ЗАПРОС НА ПОЛУЧЕНИЕ ДАННЫХ О КОНКРЕТНОЙ ВАКАНСИИ**
1. В ответе приходит словарь с 47-мью ключами: 'id', 'premium', 'billing_type', 'relations', 'name', 'insider_interview', 'response_letter_required', 'area', 'salary', 'type', 'address', 'allow_messages', 'experience', 'schedule', 'employment', 'department', 'contacts', 'description', 'branded_description', 'vacancy_constructor_template', 'key_skills', 'accept_handicapped', 'accept_kids', 'archived', 'response_url', 'specializations', 'professional_roles', 'code', 'hidden', 'quick_responses_allowed', 'driver_license_types', 'accept_incomplete_resumes', 'employer', 'published_at', 'created_at', 'initial_created_at', 'negotiations_url', 'suitable_resumes_url', 'apply_alternate_url', 'has_test', 'test', 'alternate_url', 'working_days', 'working_time_intervals', 'working_time_modes', 'accept_temporary', 'languages';
2. Ключевыми данными для нашего исследования можно считать пункты _description_ и _key\_skills_

In [64]:
import pandas as pd
import numpy as np
import requests
from datetime import datetime, timedelta
import dateparser
import json

**URL HH**

In [6]:
url_HH = 'https://api.hh.ru/vacancies'

**QUERY PARAMETRS**

In [2]:
query_params = {
    'text': 'Аналитик', 
    'area': 113, 
    'per_page': 100,
    }

In [10]:
response = requests.get(url=url_HH, params=query_params).json()

In [92]:
print(*response.keys(), sep='\n')

items
found
pages
per_page
page
clusters
arguments
alternate_url


In [14]:
len(response['items'][0].keys()) # кол-во полей в разделе items - поле из запроса, которое содержит список вакансий

31

In [12]:
response['found'] # параметр, отвечающий за кол-во вакансий в запросе, на его основе в дальнейшем пропишем if condition

907

In [90]:
print(*response['items'][0].keys(), sep='\n') # поля, которые содержатся в items

id
premium
name
department
has_test
response_letter_required
area
salary
type
address
response_url
sort_point_distance
published_at
created_at
archived
apply_alternate_url
insider_interview
url
adv_response_url
alternate_url
relations
employer
snippet
contacts
schedule
working_days
working_time_intervals
working_time_modes
accept_temporary
professional_roles
accept_incomplete_resumes


Готовим ссылку на конкретную вакансию

In [31]:
resume_url = "https://api.hh.ru/vacancies/" + response['items'][1]['id']
resume_response = requests.get(url=resume_url).json()

In [23]:
len(resume_response.keys()) # кол-во полей в запросе конкретной вакансии

47

In [32]:
# with open("2023_02_13_test_resume.json", 'w') as file:
#     json.dump(resume_response, file, indent=4, ensure_ascii=False)

In [93]:
print(*resume_response.keys(), sep='\n')

id
premium
billing_type
relations
name
insider_interview
response_letter_required
area
salary
type
address
allow_messages
experience
schedule
employment
department
contacts
description
branded_description
vacancy_constructor_template
key_skills
accept_handicapped
accept_kids
archived
response_url
specializations
professional_roles
code
hidden
quick_responses_allowed
driver_license_types
accept_incomplete_resumes
employer
published_at
created_at
initial_created_at
negotiations_url
suitable_resumes_url
apply_alternate_url
has_test
test
alternate_url
working_days
working_time_intervals
working_time_modes
accept_temporary
languages


Вводим предварительные даты исследования и преобразуем в datetime-объекты:

In [8]:
input_date1 = '13-02-2023'
input_date2 = '15-02-2023'

In [9]:
d1 = dateparser.parse(input_date1)
d2 = dateparser.parse(input_date2)

Введем функцию, которая проверяет введеные даты и возвращает список временных периодов, потому что в дальнейшем в функцию обращения к api проще передавать временной интервал

In [14]:
def period(d1, d2):
    if d1 == d2:
        d2 = (d2 + timedelta(days=1))
        return [(d1, d2)]
    return [(d1, d2)]

Следующая функция будет проверять временной интервал на предмет возможности выгрузки максимального объема вакансий (<= 2000) и будет возвращать количество получаемых вакансий по соответсвующему запросу.

In [15]:
def check_qty(date_from, date_to):
    global query_params
    query_params['date_from'] = date_from.strftime("%Y-%m-%dT%H:%M:%S")
    query_params['date_to'] = date_to.strftime("%Y-%m-%dT%H:%M:%S")
    response = requests.get(url=url_HH, params=query_params).json()
    return response['found']

Следующая функция будет проверять временной интервал на предмет возможности выгрузки максимального объема вакансий (<= 2000) и будет возвращать список временных интервалов, которые в далльнейшем будут переданы в функцию подготовки массива вакансий с определенными параметрами

In [16]:
time_periods = period(d1, d2)
def split_period(time_periods):
    new_list = []
    for item in time_periods:
        checker = check_qty(item[0], item[1])
        if checker >= 2000:
            interval = timedelta(hours=(item[1] - item[0]).total_seconds() / 3600 / 2)
            period_start = item[0]
            while period_start < item[1]:
                period_end = min(period_start + interval, item[1])
                period = [(period_start, period_end)]
                new_list += split_period(period)
                period_start = period_end
        else:
            new_list.append(item)
    return new_list

In [486]:
operation_list = split_period(time_periods)

8112
3419
1627
1792
4693
1832
2861
2324
1182
1142
537


In [505]:
df = pd.DataFrame()
def get_data(operation_list):
    global df
    global query_params
    global id_list
    for item in operation_list:
        query_params['date_from'] = item[0].strftime("%Y-%m-%dT%H:%M:%S")
        query_params['date_to'] = item[1].strftime("%Y-%m-%dT%H:%M:%S")
        print(item)
        for page in range(20):
            query_params['page'] = page
            response = requests.get(url=url_HH, params=query_params).json()
            df = pd.concat([df, pd.DataFrame(response['items'])])
    df.to_csv(f'{datetime.now().strftime("%Y-%m-%d-%H:%M")}_hh_request.csv')

In [506]:
get_data(operation_list)

(datetime.datetime(2023, 2, 13, 0, 0), datetime.datetime(2023, 2, 13, 12, 0))
(datetime.datetime(2023, 2, 13, 12, 0), datetime.datetime(2023, 2, 14, 0, 0))
(datetime.datetime(2023, 2, 14, 0, 0), datetime.datetime(2023, 2, 14, 12, 0))
(datetime.datetime(2023, 2, 14, 12, 0), datetime.datetime(2023, 2, 14, 15, 0))
(datetime.datetime(2023, 2, 14, 15, 0), datetime.datetime(2023, 2, 14, 18, 0))
(datetime.datetime(2023, 2, 14, 18, 0), datetime.datetime(2023, 2, 15, 0, 0))


In [81]:
new_df = pd.read_csv('2023-02-16-17:13_hh_request.csv')

In [83]:
id_list = set(new_df.id)

In [101]:
def vacancie_description(id_list):
    start_time = datetime.now()
    print(start_time)
    global vac_df
    counter = 1
    for vac_id in id_list:
        vac_responce = requests.get(url=url_HH + '/' + str(vac_id)).json()
        data = pd.DataFrame(vac_responce.items()).transpose()
        data.columns = data.iloc[0]
        vac_df = pd.concat([vac_df, data.drop(0)])
        counter += 1
        if counter % 100 == 0:
            print(counter)
    vac_df.to_csv(f'{datetime.now().strftime("%Y-%m-%d-%H:%M")}_list_of_vacs.csv')
    end_time = datetime.now()
    print(f"{(end_time - start_time).total_seconds()} were spend")